In [60]:
import ants
import numpy as np
import nibabel as nb
import glob, os
import shutil

from nilearn import plotting

In [66]:
# directories & files 
bids_dir = '/home/lauri/Documents/TMS-FDG/derivatives/iter6s2'
subject =  'sub-tbsfdg001'
session = 'ses-002'
pet= 'sub-tbsfdg001-ses-002_pet.nii.gz'


In [67]:
pet_dir = f'{bids_dir}/{subject}/{session}/pet'
pet_fullpath = f'{pet_dir}/{pet}'
pet= ants.image_read(pet_fullpath)


In [68]:

# do first round motion correction 
images_unmerged = ants.ndimage_to_list( pet )
motion_corrected_first = list()
for i in range(len(images_unmerged)):
    areg = ants.registration( images_unmerged[0], images_unmerged[i], type_of_transform='Affine' )
    motion_corrected_first.append( areg[ 'warpedmovout' ] )   

# sum the motion corrected as a template
template= motion_corrected_first[0]
for i in range(1,len(motion_corrected_first)):
    template + motion_corrected_first[i]
template_pet_fullpath = f'{pet_dir}/{subject}-{session}_template_pet.nii.gz' 
ants.image_write(image=template, filename=template_pet_fullpath) 

# redo motion correction to the template and this time save the warps 
regdir = f'{pet_dir}/reg'
if not os.path.isdir(regdir):
    os.makedirs(regdir)

motion_affines = []
motion_corrected_second = list()
for i in range(len(images_unmerged)):
    areg = ants.registration( template, images_unmerged[i], type_of_transform='Affine', outprefix=regdir )
    motion_affine = f'{regdir}/fwdtransform_frame{i}_Affine.mat'
    motion_affines.append(motion_affine)
    shutil.move(areg['fwdtransforms'][0], motion_affine)
    motion_corrected_second.append( areg[ 'warpedmovout' ] ) 
    
# save the motion corrected PET   
motCorr = ants.list_to_ndimage( pet, motion_corrected )
mc_pet_fullpath = f'{pet_dir}/{subject}-{session}_mc_pet.nii.gz' 
ants.image_write(image=motCorr, filename=mc_pet_fullpath) 

# sum the twice motion corrected PET 
sum_mc = motion_corrected_second[0]
for i in range(1,len(motion_corrected_second)):
    sum_mc  + motion_corrected_second[i]
sum_mc_pet_fullpath = f'{pet_dir}/{subject}-{session}_sum_mc_pet.nii.gz' 
ants.image_write(image=sum_mc, filename=sum_mc_pet_fullpath) 

# calculate the registration of sum_mc to T1w
T1w_file = glob.glob(f'{bids_dir}/{subject}/{session}/anat/*_desc-preproc_T1w.nii.gz')[0]
T1w = ants.image_read(T1w_file)

T1w_MNI_file = glob.glob(f'{bids_dir}/{subject}/{session}/anat/*_res-2_desc-preproc_T1w.nii.gz')[0]
T1w_MNI = ants.image_read(T1w_MNI_file)

T1w2MNI = glob.glob(f'{bids_dir}/{subject}/{session}/anat/*MNI152NLin6Asym_mode-image_xfm.h5')[0]

# calculate PET2T1w
registration = ants.registration(fixed=T1w, moving=sum_mc, type_of_transform='Affine',metric='MI', outprefix=regdir)
PET2T1W =f'{regdir}/fwdtransform_PET2T1w_Affine.mat'
shutil.move(registration['fwdtransforms'][0], PET2T1W)


'/home/lauri/Documents/TMS-FDG/derivatives/iter6s2/sub-tbsfdg001/ses-002/pet/reg/fwdtransform_PET2T1w_Affine.mat'

In [104]:
motion_corrected_second[0]
print(motion_corrected_second[0].origin)


(-358.8940124511719, -359.4070129394531, 128.38400268554688)


In [105]:

T1w2MNI = glob.glob(f'{bids_dir}/{subject}/{session}/anat/*MNI152NLin6Asym_mode-image_xfm.h5')[0]
PET2T1W =f'{regdir}/fwdtransform_PET2T1w_Affine.mat'
# motion_affines is a list of affines saved when motion correcting the PET 

images_unmerged = ants.ndimage_to_list( pet )
frames_in_MNI = list()

for i in range(len(images_unmerged)):
    comb_transforms = ants.apply_transforms(fixed=T1w_MNI,
                                        moving=images_unmerged[i],
                                        transformlist=[ T1w2MNI, PET2T1W, motion_affines[i]],
                                        compose=regdir)
    frame_MNI = ants.apply_transforms(fixed=T1w_MNI,moving=images_unmerged[i],transformlist=[comb_transforms])
    frames_in_MNI.append( frame_MNI )  
    

#mc_MNI_PET = ants.list_to_ndimage( T1w_MNI, frames_in_MNI )
    

In [113]:
motion_corrected_second

[ANTsImage (RAS)
 	 Pixel Type : float (float32)
 	 Components : 1
 	 Dimensions : (172, 172, 127)
 	 Spacing    : (4.1725, 4.1725, 2.031)
 	 Origin     : (-358.894, -359.407, 128.384)
 	 Direction  : [ 1.  0.  0.  0.  1.  0.  0.  0. -1.],
 ANTsImage (RAS)
 	 Pixel Type : float (float32)
 	 Components : 1
 	 Dimensions : (172, 172, 127)
 	 Spacing    : (4.1725, 4.1725, 2.031)
 	 Origin     : (-358.894, -359.407, 128.384)
 	 Direction  : [ 1.  0.  0.  0.  1.  0.  0.  0. -1.],
 ANTsImage (RAS)
 	 Pixel Type : float (float32)
 	 Components : 1
 	 Dimensions : (172, 172, 127)
 	 Spacing    : (4.1725, 4.1725, 2.031)
 	 Origin     : (-358.894, -359.407, 128.384)
 	 Direction  : [ 1.  0.  0.  0.  1.  0.  0.  0. -1.],
 ANTsImage (RAS)
 	 Pixel Type : float (float32)
 	 Components : 1
 	 Dimensions : (172, 172, 127)
 	 Spacing    : (4.1725, 4.1725, 2.031)
 	 Origin     : (-358.894, -359.407, 128.384)
 	 Direction  : [ 1.  0.  0.  0.  1.  0.  0.  0. -1.],
 ANTsImage (RAS)
 	 Pixel Type : float (

In [78]:
?ants.list_to_ndimage

Signature: ants.list_to_ndimage(image, image_list)
Docstring:
Merge list of multiple scalar ANTsImage types of dimension into one
ANTsImage of dimension plus one

ANTsR function: `mergeListToNDImage`

Arguments
---------
image : target image space
image_list : list/tuple of ANTsImage types
    scalar images to merge into target image space

Returns
-------
ANTsImage

Example
-------
>>> import ants
>>> image = ants.image_read(ants.get_ants_data('r16'))
>>> image2 = ants.image_read(ants.get_ants_data('r16'))
>>> imageTar = ants.make_image( ( *image2.shape, 2 ) )
>>> image3 = ants.list_to_ndimage( imageTar, [image,image2])
>>> image3.dimension == 3
File:      ~/.local/lib/python3.11/site-packages/ants/utils/ndimage_to_list.py
Type:      function

In [46]:
p = pet_dir + '/' + 'summed.nii.gz'
ants.image_write(summed, p) 
mc_sum_pet_fullpath = f'{pet_dir}/{subject}-{session}_mc_sum_pet.nii.gz' 
print(mc_sum_pet_fullpath)
#mc_sum_pet_fullpath = f'{pet_dir}/{subject}-{session}_mc_sum_pet_nii.gz'
#mc_sum_pet_fullpath
pet_dir = f'{bids_dir}/{subject}/{session}/pet'
os.listdir(pet_dir)
ants.image_write(summed, mc_sum_pet_fullpath) 

/home/lauri/Documents/TMS-FDG/derivatives/iter6s2/sub-tbsfdg001/ses-002/pet/sub-tbsfdg001-ses-002_mc_sum_pet_nii.gz


RuntimeError: /project/itksource/Modules/IO/ImageBase/include/itkImageFileWriter.hxx:117:
 Could not create IO object for writing file /home/lauri/Documents/TMS-FDG/derivatives/iter6s2/sub-tbsfdg001/ses-002/pet/sub-tbsfdg001-ses-002_mc_sum_pet_nii.gz
  Tried to create one of the following:
    BMPImageIO
    BioRadImageIO
    Bruker2dseqImageIO
    GDCMImageIO
    GE4ImageIO
    GE5ImageIO
    GiplImageIO
    HDF5ImageIO
    JPEGImageIO
    JPEG2000ImageIO
    LSMImageIO
    MGHImageIO
    MINCImageIO
    MRCImageIO
    MetaImageIO
    NiftiImageIO
    NrrdImageIO
    PNGImageIO
    StimulateImageIO
    TIFFImageIO
    VTKImageIO
  You probably failed to set a file suffix, or
    set the suffix to an unsupported type.


In [23]:

summed = motion_corrected[0]
for i in range(1,len(motion_corrected)):
    summed + motion_corrected[i]
mc_sum_pet_fullpath = f'{pet_dir}/{subject}-{session}_mc_sum_pet_nii.gz' 
ants.image_write(image=summed, filename=mc_sum_pet_fullpath) 

RuntimeError: /project/itksource/Modules/IO/ImageBase/include/itkImageFileWriter.hxx:117:
 Could not create IO object for writing file /home/lauri/Documents/TMS-FDG/derivatives/iter6s2/sub-tbsfdg001/ses-002/pet/sub-tbsfdg001-ses-002_mc_sum_pet_nii.gz
  Tried to create one of the following:
    BMPImageIO
    BioRadImageIO
    Bruker2dseqImageIO
    GDCMImageIO
    GE4ImageIO
    GE5ImageIO
    GiplImageIO
    HDF5ImageIO
    JPEGImageIO
    JPEG2000ImageIO
    LSMImageIO
    MGHImageIO
    MINCImageIO
    MRCImageIO
    MetaImageIO
    NiftiImageIO
    NrrdImageIO
    PNGImageIO
    StimulateImageIO
    TIFFImageIO
    VTKImageIO
  You probably failed to set a file suffix, or
    set the suffix to an unsupported type.


In [15]:
def motion_correct_warp_sum(bids_dir, subject, session, pet):
    """do motion correction, warp to MNI in one go and then sum the motion corrected images in MNI space 

    Args:
        bids_dir (str): derivatives folder 
        subject (str): subject ID
        session (str): session
        pet (str): filename
    Returns:
        full paths to mc_corrected PET file in MNI and mc_corrected summed PET file in MNI
    """
    
    pet_dir = f'{bids_dir}/{subject}/{session}/pet'
    pet_fullpath = f'{pet_dir}/{pet}'
    pet= ants.image_read(pet_fullpath)
    
    images_unmerged = ants.ndimage_to_list( pet )
    motion_corrected = list()
    for i in range(len(images_unmerged)):
        areg = ants.registration( images_unmerged[0], images_unmerged[i], type_of_transform='Affine' )
        motion_corrected.append( areg[ 'warpedmovout' ] )    
    motCorr = ants.list_to_ndimage( pet, motion_corrected )
    mc_pet_fullpath = f'{pet_dir}/{subject}-{session}_mc_pet_nii.gz' 
    ants.image_write(image=motCorr, filename=mc_pet_fullpath) 
    
    summed = motion_corrected[0]
    for i in range(1,len(motion_corrected)):
        summed + motion_corrected[i]
    mc_sum_pet_fullpath = f'{pet_dir}/{subject}-{session}_mc_sum_pet_nii.gz' 
    ants.image_write(image=summed, filename=mc_sum_pet_fullpath) 

    return(mc_pet_fullpath, mc_sum_pet_fullpath)


In [22]:
def register(bids_dir, subject, session, pet):
    """Register and write PET image to MNI152 space by combining coregisteration of the PET to T1 and T1 to MNI152

    Args:
        bids_dir (str): derivatives folder 
        subject (str): subject ID
        session (str): session
        pet (str): filename
    Returns:
    Full path to MNI152 registered summed PET image
    """
    pet_dir = f'{bids_dir}/{subject}/{session}/pet'
    pet_fullpath = f'{pet_dir}/{pet}'
    pet= ants.image_read(pet_fullpath)
    
    T1w_file = glob.glob(f'{bids_dir}/{subject}/{session}/anat/*_desc-preproc_T1w.nii.gz')[0]
    T1w = ants.image_read(T1w_file)
    
    T1w_MNI_file = glob.glob(f'{bids_dir}/{subject}/{session}/anat/*_res-2_desc-preproc_T1w.nii.gz')[0]
    T1w_MNI = ants.image_read(T1w_MNI_file)
    
    T1w2MNI = glob.glob(f'{bids_dir}/{subject}/{session}/anat/*MNI152NLin6Asym_mode-image_xfm.h5')[0]
    
    # calculate PET2T1w
    regdir = f'{pet_dir}/antsr'
    os.
    registration = ants.registration(fixed=T1w, moving=pet, type_of_transform='Affine',metric='MI', outprefix=pet_dir)
    
    comptx_file = ants.apply_transforms(fixed=T1w_MNI,
                                    moving=pet,
                                    transformlist=[ T1w2MNI, registration['fwdtransforms'][0]],
                                    compose=pet_dir)

# apply PET to MNI transform to dynamic_template
PET_MNIspace = ants.apply_transforms(fixed=T1w_MNI,moving=SUVR_pet,transformlist=[comptx_file])
#ants.image_write(template_MNIspace,os.path.join(REGDIR,'dynamic_template_MNI.nii.gz'))
ants.image_write(PET_MNIspace,f'{bids_dir}/{subject}/{session}/pet/SUVR_in_MNI.nii.gz')
    #mywarpedimage = ants.apply_transforms(fixed=T1w, moving=pet,type_of_transform='Affine',
    #                                  transformlist=registration['fwdtransforms'])
    
    

SyntaxError: invalid syntax (650549416.py, line 26)

In [ ]:
# motion correct 
images_unmerged = ants.ndimage_to_list( pet )
motion_corrected = list()
for i in range( len( images_unmerged ) ):
    areg = ants.registration( images_unmerged[0], images_unmerged[i], "SyN" )
    motion_corrected.append( areg[ 'warpedmovout' ] )
# sum

# register 

# calculate SUVR 


In [3]:
?ants.set_direction

Signature: ants.set_direction(image, direction)
Docstring:
Set direction of ANTsImage

ANTsR function: `antsSetDirection`
File:      ~/.local/lib/python3.11/site-packages/ants/core/ants_image.py
Type:      function